## PGA Draftkings Notebook
Use [PGA Website](https://www.pgatour.com/tournaments/schedule.html) to look up tournament info and fill out first USER INPUT block below. (Fetch/XHR from Network tab, Paylod sub-tab)

Looks like the tournament ID is also in the address bar.

### User Input

In [1]:
# === USER INPUTS ===
# Old Tournament
old_tournament_name = "U.S. Open"
tournament_date = "6/15/2025"  # Ending date of tournament
old_course = "Oakmont Country Club"
tournament_id = "R2025026"  # Tournament ID from the PGA Tour API - Also visible in the URL of the tournament page

# New Tournament
new_tournament_name = "Travelers Championship"
new_ending_date = "6/22/2025"
new_course = "TPC River Highlands"
new_season = 2025


# === LIBRARIES AND VARIABLES ===
# Import necessary libraries
import requests
import pandas as pd
from datetime import datetime
import sqlite3 as sql
import numpy as np
from numpy import nan
import os
import importlib
import utils.db_utils
from utils.db_utils import TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP

# === TOURNAMENT CONFIG ===
tournament_config = {
    "old": {
        "name": old_tournament_name,
        "date": tournament_date,
        "course": old_course,
        "id": tournament_id
    },
    "new": {
        "name": new_tournament_name,
        "course": new_course,
        "season": new_season,
        "ending_date": pd.to_datetime(new_ending_date, format="%m/%d/%Y"),
        "quoted_course": f'"{new_course}"',
        "quoted_name": f"'{new_tournament_name}'"
    }
}

this_week_key = str(tournament_config["new"]["ending_date"].date())

# Create a minimal one-row history DataFrame
this_week_history = pd.DataFrame([{
    "SEASON": tournament_config["new"]["season"],
    "TOURNAMENT": tournament_config["new"]["name"],
    "ENDING_DATE": tournament_config["new"]["ending_date"],
    "COURSE": tournament_config["new"]["course"],
    "TOURN_ID": None
}])

### Update Database

#### Old Tournament

In [2]:
importlib.reload(utils.db_utils)  # Only needed if you're actively editing db_utils.py
from utils.db_utils import update_tournament_results

# Change these each year!!
season = 2025
year = 20250  # Unique GraphQL year distinguishing number in case of multiple per year

# Run the update
db_path = "data/golf.db"  # Or use os.path.join("data", "golf.db")
tournDf = update_tournament_results(tournament_config, db_path, season, year)

# Show just the most recent tournament added for confirmation
from sqlalchemy import create_engine

engine = create_engine(f"sqlite:///{db_path}")

query = f"""
SELECT *
FROM tournaments
WHERE TOURN_ID = '{tournament_config['old']['id']}'
  AND ENDING_DATE = '{datetime.strptime(tournament_config['old']['date'], '%m/%d/%Y').date()}'
"""

recent = pd.read_sql(query, engine)
engine.dispose()
recent.sort_values(by='FINAL_POS').head()

📦 Fetching results for tournament ID R2025026 (U.S. Open), year: 20250
✅ 156 new rows added for 'U.S. Open'


,SEASON,ENDING_DATE,TOURN_ID,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,ROUNDS:1,ROUNDS:2,ROUNDS:3,ROUNDS:4,OFFICIAL_MONEY,FEDEX_CUP_POINTS
53,2025,2025-06-15,R2025026,U.S. Open,Oakmont Country Club,J.J. Spaun,1,1,-4,+2,-1,+2,"$4,300,000.00",750.000
116,2025,2025-06-15,R2025026,U.S. Open,Oakmont Country Club,Robert MacIntyre,2,2,E,+4,-1,-2,"$2,322,000.00",500.000
148,2025,2025-06-15,R2025026,U.S. Open,Oakmont Country Club,Viktor Hovland,3,3,+1,-2,E,+3,"$1,462,525.00",350.000
22,2025,2025-06-15,R2025026,U.S. Open,Oakmont Country Club,Cameron Young,T4,4,E,+4,-1,E,"$878,815.00",300.000
146,2025,2025-06-15,R2025026,U.S. Open,Oakmont Country Club,Tyrrell Hatton,T4,4,+3,E,-2,+2,"$878,815.00",0.000


#### Stats

In [3]:
importlib.reload(utils.db_utils)
from utils.db_utils import update_season_stats  # <- This line is essential

# Change these each year!!
statsYear = 2025

stats_df = update_season_stats(statsYear, db_path)
stats_df.head()

✅ Overwrote stats for season 2025 with 998 rows.


,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,...,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON
0,A.J. Ewart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,606,.1950,2025
1,Aaron Baddeley,165.0,-1.022,175.0,-1.292,141.0,-0.265,4.0,0.536,158.0,...,289.6,112.0,57.29%,169.0,60.48%,79.0,60.24%,451,.2886,2025
2,Aaron Cockerill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,353,.3962,2025
3,Aaron Rai,22.0,0.815,25.0,0.401,31.0,0.414,97.0,0.001,125.0,...,286.9,1.0,73.55%,18.0,69.12%,123.0,57.79%,32,2.6753,2025
4,Aaron Wilkin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,814,.1192,2025


#### Odds
Not usually needed for weekly routine.

**Manual Fix! Odds name cleanup (only needed when joins fail)**

Make sure to update the dictionaries in db_utils.py if new names need to be added.

In [30]:
importlib.reload(utils.db_utils)
from utils.db_utils import clean_odds_names, PLAYER_NAME_MAP, TOURNAMENT_NAME_MAP

db_path = "data/golf.db" 
updated_odds = clean_odds_names(db_path, TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP)
updated_odds.head()

ℹ️ No odds rows required name cleanup.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS,TOURNAMENT_ORIG,PLAYER_ORIG


**Historical Odds Updates**

Only run this when loading in entire year odds at the start of each year or if corrections need to be made.  This will load in the entire year into the database and update it using the dictionary in db_utils.py.

In [31]:
importlib.reload(utils.db_utils)
from utils.db_utils import import_historical_odds

oddsYear = "2022-2023"    # URL segment
season = 2023             # PGA Tour season
db_path = "data/golf.db"

odds_df = import_historical_odds(oddsYear, season, db_path)
odds_df.head()

ℹ️ Historical odds for season 2023 already exist — no new rows added.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
4,2023,Fortinet Championship,2022-09-18,Hideki Matsuyama,18/1,18.0
5,2023,Fortinet Championship,2022-09-18,Max Homa,18/1,18.0
6,2023,Fortinet Championship,2022-09-18,Corey Conners,20/1,20.0
7,2023,Fortinet Championship,2022-09-18,Maverick McNealy,25/1,25.0
8,2023,Fortinet Championship,2022-09-18,Taylor Pendrith,25/1,25.0


**Not normally needed**

The code below is a way to troubleshoot the odds function that is in db_utils.py.  If it does not pull the stats correctly, we can run it outside of the db_utils.py file and explore what might be wrong with it, iterating until it works, and then using that fix to update db_utils.py.  This is not normally needed, but wanted to leave this in place as a reference because this odds website can be tricky.

In [32]:
import pandas as pd
import numpy as np
import requests
import re
from datetime import datetime
from io import StringIO

# === USER INPUT ===
oddsYear = "2020-2021"    # URL segment
season = 2021        # PGA Tour season

url = f"http://golfodds.com/archives-{oddsYear}.html"
response = requests.get(url)
tables = pd.read_html(StringIO(response.text))
# raw_df = tables[5]  # the actual table of interest
# Find the largest 2-column table that contains at least some odds-like strings
raw_df = None
for tbl in tables:
    if tbl.shape[1] == 2 and tbl.shape[0] > 50:  # Rough filter
        sample = tbl.iloc[:, 1].astype(str).str.contains(r"\d+/\d+").sum()
        if sample > 5:
            raw_df = tbl
            break

if raw_df is None:
    raise ValueError("❌ Could not find valid odds table on the page.")

# === STEP 1: Initial clean-up ===
df = raw_df.dropna(how="all").reset_index(drop=True)
df.columns = ["PLAYER", "ODDS"]

# 🔧 Clean up non-breaking spaces and extra whitespace
df["PLAYER"] = (
    df["PLAYER"]
    .astype(str)
    .str.replace("\xa0", " ", regex=False)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

df.insert(loc=0, column="SEASON", value=season)
df.insert(loc=1, column="TOURNAMENT", value=np.nan)
df.insert(loc=2, column="ENDING_DATE", value=np.nan)

# === STEP 2: Helper function for parsing date strings ===
def parse_ending_date(text):
    import re
    from datetime import datetime

    # Normalize whitespace and symbols
    text = (
        text.replace("\u2013", "-")
            .replace("–", "-")
            .replace("\xa0", " ")
    )
    text = re.sub(r"\bSept(?!ember)\b", "Sep", text)

    # ✅ Fix typo: "Match" → "March" only when used in a date context
    text = re.sub(r"\bMatch(?=\s+\d{1,2}\s*[-–]\s*\d{1,2},\s*\d{4})", "March", text)

    # Pattern 1: "July 30 - August 2, 2015" or "Oct 29 - Nov 1, 2015"
    match = re.search(r"(\w+)\s\d+\s*-\s*(\w+)\s(\d+),\s(\d{4})", text)
    if match:
        month2, day2, year = match.group(2), match.group(3), match.group(4)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month2} {day2}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 2: "November 21-24, 2024"
    match = re.search(r"(\w+)\s\d+-\d+,\s(\d{4})", text)
    if match:
        month, year = match.group(1), match.group(2)
        day = re.search(r"(\d+)-(\d+)", text).group(2)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month} {day}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 3: "Sunday, October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%A, %B %d, %Y").date()
    except ValueError:
        pass

    # Pattern 4: "October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%B %d, %Y").date()
    except ValueError:
        pass

    return None

last_tourn_name = None
last_date = None

# === STEP 3: Iterate block by block ===
final_rows = []
i = 0
last_tourn_name = None
last_end_date = None

while i < len(df) - 4:
    player_i = str(df.loc[i, "PLAYER"])
    player_i2 = str(df.loc[i + 2, "PLAYER"])
    player_i3 = str(df.loc[i + 3, "PLAYER"]).lower()

    # Detect start of a new tournament block
    is_header = (
        pd.isna(df.loc[i, "ODDS"]) and
        pd.isna(df.loc[i + 1, "ODDS"]) and (
            re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", player_i2) or
            re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", player_i2)
        )
    )

    if is_header:
        tourn_name = player_i.strip()
        end_date = parse_ending_date(player_i2)

        # Skip cancelled or empty blocks
        if "cancelled" in player_i3:
            print(f"⚠️ Skipping cancelled tournament: {tourn_name} — {end_date}")
            i += 4
            continue

        # Avoid duplicate block processing
        if tourn_name == last_tourn_name and end_date == last_end_date:
            i += 1
            continue

        print(f"📍 Detected: {tourn_name} — Ending: {end_date}")
        last_tourn_name = tourn_name
        last_end_date = end_date
        i += 4  # Skip header lines

        # Collect all player rows until next header block
        while i < len(df) - 2:
            next_i2 = str(df.loc[i + 2, "PLAYER"])
            is_next_header = (
                pd.isna(df.loc[i, "ODDS"]) and
                pd.isna(df.loc[i + 1, "ODDS"]) and (
                    re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", next_i2) or
                    re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", next_i2)
                )
            )
            if is_next_header:
                break

            if pd.notna(df.loc[i, "ODDS"]):
                row = df.loc[i].copy()
                row["TOURNAMENT"] = tourn_name
                row["ENDING_DATE"] = end_date
                final_rows.append(row)
            i += 1
    else:
        i += 1

# === STEP 4: Create cleaned DataFrame ===
clean_df = pd.DataFrame(final_rows)

# ✅ Prevent crash if nothing was parsed
if clean_df.empty or "PLAYER" not in clean_df.columns:
    print(f"⚠️ No valid tournament blocks detected for season {season} ({oddsYear})")
    final_df = pd.DataFrame()  # Safe fallback
else:
    # Remove winner tag
    clean_df["PLAYER"] = clean_df["PLAYER"].str.replace(r"\s\*Winner\*", "", regex=True)

    # Clean odds to numeric
    clean_df["VEGAS_ODDS"] = (
        clean_df["ODDS"]
        .str.replace(",", "")
        .str.extract(r"(\d+)/(\d+)")
        .astype(float)
        .apply(lambda x: x[0] / x[1], axis=1)
    )

    # Final output with source index for debugging
    final_df = clean_df[
        ["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER", "ODDS", "VEGAS_ODDS"]
    ].reset_index(drop=True)

    # Drop non-standard team events (e.g., Presidents Cup, Ryder Cup)
    drop_terms = ["Presidents Cup", "Ryder Cup"]
    final_df = final_df[~final_df["TOURNAMENT"].str.contains("|".join(drop_terms), case=False, na=False)]

    display(final_df.head())



📍 Detected: Safeway Open — Ending: 2020-09-13
📍 Detected: US Open — Ending: 2020-09-20
📍 Detected: R & C Championship — Ending: 2020-09-27
📍 Detected: at Big Cedar Lodge - — Ending: 2020-09-22
📍 Detected: Sanderson Farms Champ — Ending: 2020-10-04
📍 Detected: Shriners H for C Open — Ending: 2020-10-11
📍 Detected: The CJ Cup — Ending: 2020-10-18
📍 Detected: ZOZO CHAMPIONSHIP — Ending: 2020-10-25
📍 Detected: Bermuda Championship — Ending: 2020-11-01
📍 Detected: Vivint Houston Open — Ending: 2020-11-08
📍 Detected: The Masters — Ending: 2020-11-15
📍 Detected: The RSM Classic — Ending: 2020-11-22
📍 Detected: Champions for Change — Ending: 2020-11-27
📍 Detected: Mayakoba Golf Classic — Ending: 2020-12-06
📍 Detected: QBE Shootout — Ending: 2020-12-13
📍 Detected: Sentry Tourn of Champions — Ending: 2021-01-10
📍 Detected: Sony Open in Hawaii — Ending: 2021-01-17
📍 Detected: The American Express — Ending: 2021-01-24
📍 Detected: Abu Dhabi HSBC Champ — Ending: 2021-01-24
📍 Detected: Farmers Insura

,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
0,2021,Safeway Open,2020-09-13,Phil Mickelson,20/1,20.0
1,2021,Safeway Open,2020-09-13,Si Woo Kim,20/1,20.0
2,2021,Safeway Open,2020-09-13,Brendan Steele,20/1,20.0
3,2021,Safeway Open,2020-09-13,Shane Lowry,25/1,25.0
4,2021,Safeway Open,2020-09-13,Sergio Garcia,30/1,30.0


In [33]:
from datetime import datetime, date
# ✅ Check for non-date types in ENDING_DATE
non_dates = final_df[~final_df["ENDING_DATE"].apply(lambda x: isinstance(x, date))]

print(f"🧪 Rows with invalid ENDING_DATE values: {len(non_dates)}")
display(non_dates.head(10))



🧪 Rows with invalid ENDING_DATE values: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


In [34]:
dupes = final_df.duplicated(subset=["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER"], keep=False)

print(f"🚨 Duplicate primary keys in final_df: {dupes.sum()}")
display(final_df[dupes].sort_values(by=["SEASON", "TOURNAMENT", "PLAYER"]))

🚨 Duplicate primary keys in final_df: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


## Historical Data

### Pull Relevant Seasons
Do a check to see when this course or tournament have been historically played.

In [4]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_combined_history_seasons

# === USER INPUT ===
seasons = list(range(2016, 2025))  # Adjust as needed
db_path = "data/golf.db"

# Pull course and tournament from config
n_course = tournament_config["new"]["course"]
n_tourn = tournament_config["new"]["name"]

# Fetch relevant history
history_df = get_combined_history_seasons(db_path, course=n_course, tournament=n_tourn, allowed_seasons=seasons)
history_df.head(20)


ℹ️ Found 9 relevant tournaments from course or tournament name.


,SEASON,COURSE,TOURN_ID,TOURNAMENT,ENDING_DATE
780,2016,TPC River Highlands,034,Travelers Championship,2016-08-07
624,2017,TPC River Highlands,034,Travelers Championship,2017-06-25
468,2018,TPC River Highlands,034,Travelers Championship,2018-06-24
312,2019,TPC River Highlands,034,Travelers Championship,2019-06-23
156,2020,TPC River Highlands,034,Travelers Championship,2020-06-28
0,2021,TPC River Highlands,034,Travelers Championship,2021-06-27
1092,2022,TPC River Highlands,034,Travelers Championship,2022-06-26
1247,2023,TPC River Highlands,R2023034,Travelers Championship,2023-06-25
1403,2024,TPC River Highlands,R2024034,Travelers Championship,2024-06-23


### Cut Percentage and FedEx Points
Use a rolling-window approach to look at the most recent cut percentage and how many FedEx cup points have been accumulated recently. This will intentionally not match the PGA Tour stats that start over every year, but will have the same amount of data all the time.  We also add a new feature called Form Density which divides the FedEx Cup Points by the Total Events.

In [5]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts = get_cut_and_fedex_history("data/golf.db", history_df, window_months=9)
# cuts["2024-05-12"].head(20)

for end_date, df in cuts.items():
    print(f"\n📆 {end_date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2016-08-07 — Travelers Championship (523 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,17,10,383.5,58.8,22.56,1,2016-08-07,Travelers Championship
1,Aaron Wise,1,0,0.0,0.0,0.00,0,2016-08-07,Travelers Championship
2,Abraham Ancer,9,2,43.0,22.2,4.78,0,2016-08-07,Travelers Championship



📆 2017-06-25 — Travelers Championship (450 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,15,7,228.29,46.7,15.22,0,2017-06-25,Travelers Championship
1,Aaron Rai,1,0,0.00,0.0,0.00,0,2017-06-25,Travelers Championship
2,Aaron Wise,6,4,0.00,66.7,0.00,4,2017-06-25,Travelers Championship



📆 2018-06-24 — Travelers Championship (458 players)


...

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,A.J. Ewart,1,0,0.00,0.0,0.00,0,2022-06-26,Travelers Championship
1,Aaron Baddeley,4,1,39.46,25.0,9.86,0,2022-06-26,Travelers Championship
2,Aaron Beverly,1,0,0.00,0.0,0.00,0,2022-06-26,Travelers Championship



📆 2023-06-25 — Travelers Championship (539 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,14,10,325.286,71.4,23.23,0,2023-06-25,Travelers Championship
1,Aaron Beverly,1,1,0.000,100.0,0.00,1,2023-06-25,Travelers Championship
2,Aaron Cockerill,1,0,0.000,0.0,0.00,0,2023-06-25,Travelers Championship



📆 2024-06-23 — Travelers Championship (502 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,13,9,181.975,69.2,14.00,0,2024-06-23,Travelers Championship
1,Aaron Cockerill,1,0,0.000,0.0,0.00,0,2024-06-23,Travelers Championship
2,Aaron Rai,16,13,544.507,81.2,34.03,7,2024-06-23,Travelers Championship


### Recent Form

In [6]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish("data/golf.db", history_df, window_months=9)

# Example preview
for date, df in recent_form.items(): 
    print(f"\n📆 {date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2016-08-07 — Travelers Championship (523 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Lee McCoy,1,4.0,5.77,2016-08-07,Travelers Championship
1,Tyrrell Hatton,2,7.5,6.83,2016-08-07,Travelers Championship
2,Jared du Toit,1,9.0,12.98,2016-08-07,Travelers Championship



📆 2017-06-25 — Travelers Championship (450 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Keith Mitchell,1,11.0,15.87,2017-06-25,Travelers Championship
1,Oscar Fraustro,1,13.0,18.76,2017-06-25,Travelers Championship
2,Cam Davis,1,15.0,21.64,2017-06-25,Travelers Championship



📆 2018-06-24 — Travelers Championship (458 players)


...

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Haotong Li,1,12.0,17.31,2022-06-26,Travelers Championship
1,Martin Contini,1,16.0,23.08,2022-06-26,Travelers Championship
2,Rory McIlroy,11,16.9,6.80,2022-06-26,Travelers Championship



📆 2023-06-25 — Travelers Championship (539 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Joohyung Kim,1,1.0,1.44,2023-06-25,Travelers Championship
1,Brooks Koepka,3,6.7,4.83,2023-06-25,Travelers Championship
2,Scottie Scheffler,17,7.8,2.70,2023-06-25,Travelers Championship



📆 2024-06-23 — Travelers Championship (502 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Bryson DeChambeau,3,3.0,2.16,2024-06-23,Travelers Championship
1,Scottie Scheffler,15,6.7,2.42,2024-06-23,Travelers Championship
2,Xander Schauffele,15,12.9,4.65,2024-06-23,Travelers Championship


### Course History

In [7]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

# Filter history_df for only the course we're targeting
target_course = tournament_config["new"]["course"]
course_df = history_df[history_df["COURSE"] == target_course]
course_hist = get_course_history("data/golf.db", course_df)

# View example
for date, df in course_hist.items():
    if not df.empty:
        print(f"\n🏌️‍♂️ Course history for {df['TOURNAMENT'].iloc[0]} on {date}")
        display(df.head(3))


🏌️‍♂️ Course history for Travelers Championship on 2016-08-07


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,1,48.0,69.25,2016-08-07,TPC River Highlands,Travelers Championship
1,Adam Hadwin,1,90.0,129.84,2016-08-07,TPC River Highlands,Travelers Championship
2,Alex Cejka,1,90.0,129.84,2016-08-07,TPC River Highlands,Travelers Championship



🏌️‍♂️ Course history for Travelers Championship on 2017-06-25


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,2,41.0,37.32,2017-06-25,TPC River Highlands,Travelers Championship
1,Abraham Ancer,1,56.0,80.79,2017-06-25,TPC River Highlands,Travelers Championship
2,Adam Hadwin,2,90.0,81.92,2017-06-25,TPC River Highlands,Travelers Championship



🏌️‍♂️ Course history for Travelers Championship on 2018-06-24


...

,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,5,70.4,39.29,2022-06-26,TPC River Highlands,Travelers Championship
1,Aaron Wise,2,56.0,50.97,2022-06-26,TPC River Highlands,Travelers Championship
2,Abraham Ancer,5,33.8,18.86,2022-06-26,TPC River Highlands,Travelers Championship



🏌️‍♂️ Course history for Travelers Championship on 2023-06-25


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,4,76.0,47.22,2023-06-25,TPC River Highlands,Travelers Championship
1,Aaron Rai,1,40.0,57.71,2023-06-25,TPC River Highlands,Travelers Championship
2,Aaron Wise,3,67.3,48.55,2023-06-25,TPC River Highlands,Travelers Championship



🏌️‍♂️ Course history for Travelers Championship on 2024-06-23


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,3,90.0,64.92,2024-06-23,TPC River Highlands,Travelers Championship
1,Aaron Rai,2,32.0,29.13,2024-06-23,TPC River Highlands,Travelers Championship
2,Aaron Wise,3,67.3,48.55,2024-06-23,TPC River Highlands,Travelers Championship


## Training Dataset

In [8]:
pd.set_option("display.max_columns", None)   # Show all columns
importlib.reload(utils.db_utils)
from utils.db_utils import build_training_rows
training_df = build_training_rows(
    db_path,
    history_df,
    cuts,
    recent_form,
    course_hist,
)
training_df.head(10)
# training_df.info()
# training_df.isna().sum().sort_values(ascending=False)

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2016,2016-08-07,Travelers Championship,TPC River Highlands,Aaron Baddeley,T34,34,146.0,-0.330,159.0,-0.366,164.0,-0.362,10.0,0.402,8.0,0.586,59.0,20.72,78.0,3.06,14.0,4.01,50.0,4.64,159.0,237.0,82.0,291.7,155.0,55.42,154.0,63.11,5.0,64.3,143.0,1.14,60.0,58.8,383.50,22.56,1.0,55.8,19.31,48.0,69.25,0
1,2016,2016-08-07,Travelers Championship,TPC River Highlands,Abraham Ancer,T56,56,144.0,-0.307,111.0,-0.024,161.0,-0.340,77.0,0.057,172.0,-0.421,154.0,18.63,120.0,3.08,136.0,4.07,183.0,4.80,131.0,205.0,178.0,276.4,27.0,65.42,174.0,61.81,131.0,56.67,619.0,0.25,NaN,22.2,43.00,4.78,0.0,80.1,34.79,NaN,NaN,0
2,2016,2016-08-07,Travelers Championship,TPC River Highlands,Adam Hadwin,CUT,90,111.0,-0.011,55.0,0.223,121.0,-0.055,132.0,-0.081,11.0,0.530,22.0,21.74,16.0,3.01,48.0,4.03,68.0,4.66,35.0,141.0,92.0,290.1,49.0,63.62,83.0,66.07,97.0,58.27,193.0,0.91,NaN,77.8,475.17,26.40,7.0,47.9,16.27,90.0,129.84,0
3,2016,2016-08-07,Travelers Championship,TPC River Highlands,Adam Rainaud,CUT,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,2016,2016-08-07,Travelers Championship,TPC River Highlands,Alex Cejka,T11,11,39.0,0.631,148.0,-0.292,12.0,0.626,20.0,0.297,90.0,0.010,17.0,21.89,3.0,2.98,14.0,4.01,10.0,4.57,78.0,177.0,155.0,281.5,22.0,65.89,39.0,67.99,3.0,65.03,130.0,1.22,NaN,60.0,445.75,29.72,2.0,53.0,19.12,90.0,129.84,1
5,2016,2016-08-07,Travelers Championship,TPC River Highlands,Andres Gonzales,T17,17,160.0,-0.469,141.0,-0.222,134.0,-0.112,151.0,-0.135,94.0,0.007,91.0,20.01,78.0,3.06,162.0,4.09,80.0,4.67,145.0,224.0,59.0,294.7,165.0,54.55,81.0,66.17,148.0,55.79,270.0,0.66,NaN,35.3,52.80,3.11,0.0,80.2,27.75,90.0,129.84,1
6,2016,2016-08-07,Travelers Championship,TPC River Highlands,Andrew Landry,CUT,90,172.0,-0.694,11.0,0.594,181.0,-0.892,179.0,-0.396,4.0,0.654,160.0,18.48,136.0,3.09,34.0,4.02,169.0,4.76,5.0,85.0,59.0,294.7,26.0,65.55,117.0,64.63,28.0,61.86,482.0,0.37,NaN,40.0,79.29,7.93,0.0,75.1,31.32,NaN,NaN,0
7,2016,2016-08-07,Travelers Championship,TPC River Highlands,Andrew Loupe,CUT,90,150.0,-0.395,105.0,-0.005,154.0,-0.281,158.0,-0.172,38.0,0.280,12.0,22.74,78.0,3.06,177.0,4.11,13.0,4.58,101.0,189.0,5.0,309.3,184.0,45.56,148.0,63.41,175.0,52.83,194.0,0.91,NaN,52.9,420.25,24.72,0.0,60.7,21.00,90.0,129.84,0
8,2016,2016-08-07,Travelers Championship,TPC River Highlands,Ben Crane,CUT,90,115.0,-0.028,117.0,-0.063,99.0,0.101,49.0,0.173,46.0,0.230,128.0,19.23,78.0,3.06,48.0,4.03,100.0,4.69,138.0,216.0,162.0,280.3,54.0,63.18,143.0,63.6,31.0,61.65,371.0,0.50,NaN,56.2,359.50,22.47,3.0,56.1,19.80,NaN,NaN,0
9,2016,2016-08-07,Travelers Championship,TPC River Highlands,Ben Martin,CUT,90,126.0,-0.117,79.0,0.079,101.0,0.081,169.0,-0.277,70.0,0.140,29.0,21.53,146.0,3.10,97.0,4.05,28.0,4.61,71.0,174.0,108.0,288.8,66.0,62.14,64.0,66.73,160.0,54.84,123.0,1.27,NaN,64.3,275.05,19.65,0.0,57.0,21.05,NaN,NaN,0


In [ ]:
# Play with different years to see if I notice anything wrong (check Odds)
training_df[training_df["SEASON"] == 2024].head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
1247,2024,2024-06-23,Travelers Championship,TPC River Highlands,Adam Hadwin,T27,27,112.0,-0.036,101.0,0.016,113.0,-0.057,102.0,0.005,82.0,0.058,63.0,23.00%,52.0,3.02,131.0,4.04,116.0,4.61,123.0,198.0,124.0,297.9,74.0,61.57%,152.0,64.30%,151.0,55.81%,56.0,2.0027,150.0,70.6,1029.292,60.55,0.0,46.0,15.91,69.8,43.37,0
1248,2024,2024-06-23,Travelers Championship,TPC River Highlands,Adam Schenk,T55,55,146.0,-0.421,53.0,0.212,166.0,-0.520,130.0,-0.114,85.0,0.051,132.0,21.24%,152.0,3.09,131.0,4.04,116.0,4.61,102.0,180.0,82.0,302.4,98.0,59.93%,149.0,64.50%,74.0,60.50%,82.0,1.4461,300.0,63.6,568.417,25.84,0.0,52.7,16.81,90.0,46.25,0
1249,2024,2024-06-23,Travelers Championship,TPC River Highlands,Adam Scott,T39,39,38.0,0.540,33.0,0.322,87.0,0.114,67.0,0.104,29.0,0.398,79.0,22.54%,38.0,3.01,57.0,4.00,44.0,4.54,75.0,170.0,49.0,307.1,121.0,58.28%,102.0,66.35%,45.0,62.09%,20.0,3.2584,100.0,86.7,672.759,44.85,3.0,33.3,12.01,16.0,14.56,0
1250,2024,2024-06-23,Travelers Championship,TPC River Highlands,Adam Svensson,T16,16,33.0,0.592,99.0,0.019,28.0,0.380,39.0,0.193,164.0,-0.453,153.0,20.41%,142.0,3.08,85.0,4.01,121.0,4.62,62.0,158.0,118.0,298.3,40.0,64.18%,88.0,67.13%,60.0,61.39%,112.0,1.1671,150.0,79.2,374.841,15.62,10.0,47.7,14.82,68.3,49.27,1
1251,2024,2024-06-23,Travelers Championship,TPC River Highlands,Akshay Bhatia,T5,5,56.0,0.403,58.0,0.188,51.0,0.264,119.0,-0.048,34.0,0.345,87.0,22.35%,52.0,3.02,85.0,4.01,77.0,4.57,28.0,133.0,104.0,299.8,29.0,65.50%,114.0,66.02%,54.0,61.79%,29.0,2.7339,60.0,75.0,1343.276,55.97,3.0,40.4,12.55,NaN,NaN,1


### Normalization
***Fix Historical Odds***

Many names do not have historical odds but rather are part of the "field." So giving these "NaN" values the average odds (as I would for missing stats) isn't a good approximation of reality – they are typically the "field" because they are not notable and have poor odds individually.

There are also some crazy odds numbers occasionally (1000/1 or 3000/1) that don't happen all that much and are making that tail too long.

This cell assigns odds of 1000/1 for anyone missing and clips anything larger to 1000/1 to normalize the data better and assign the missing odds more appropriately where they belong.

***OWGR Adjustment***

The NaN's for OWGR are similarly bad players.  So we should assign these the worst ranking instead of the mean and clip it at 1000 just in case there are ever outliers.

For the OWGR score, it should match the lowest score in the dataset to assign that person the same as the worst.

***Recent Form Adjustment***

The NaN's for Recent Form mean that this player has not played any tournaments in the lookback period (9 months).  Similar to Odds and OWGR, I want to punish those that don't play often in my model.  Set these to 90 (i.e. like they miss a lot of cuts - because if they aren't playing they aren't good enough to make it to these tournaments very frequently).  The adj_form feature will also need to be updated here with the new data.

***FedEx Cup Point Adjustment***

The NaN's for FedEx Cup Points means there is no data.  Therefore they should be 0.

In [10]:
# === VEGAS ODDS ===
training_df["VEGAS_ODDS"] = training_df["VEGAS_ODDS"].fillna(1000).clip(upper=1000)

# === OWGR and OWGR_RANK ===
owgr_min = training_df["OWGR"].min(skipna=True)
training_df["OWGR"] = training_df["OWGR"].fillna(owgr_min)
training_df["OWGR_RANK"] = training_df["OWGR_RANK"].fillna(1000).astype(float).clip(upper=1000)

# === RECENT FORM and adj_form ===
training_df["RECENT_FORM"] = training_df["RECENT_FORM"].fillna(90)
if "TOTAL_EVENTS_PLAYED" in training_df.columns:
    training_df["adj_form"] = (
        training_df["RECENT_FORM"] / np.log1p(training_df["TOTAL_EVENTS_PLAYED"])
    ).round(2)

# === FEDEX CUP POINTS ===
training_df["FEDEX_CUP_POINTS"] = training_df["FEDEX_CUP_POINTS"].fillna(0)

training_df.head(5)


,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2016,2016-08-07,Travelers Championship,TPC River Highlands,Aaron Baddeley,T34,34,146.0,-0.330,159.0,-0.366,164.0,-0.362,10.0,0.402,8.0,0.586,59.0,20.72,78.0,3.06,14.0,4.01,50.0,4.64,159.0,237.0,82.0,291.7,155.0,55.42,154.0,63.11,5.0,64.3,143.0,1.14,60.0,58.8,383.50,22.56,1.0,55.8,19.31,48.0,69.25,0
1,2016,2016-08-07,Travelers Championship,TPC River Highlands,Abraham Ancer,T56,56,144.0,-0.307,111.0,-0.024,161.0,-0.340,77.0,0.057,172.0,-0.421,154.0,18.63,120.0,3.08,136.0,4.07,183.0,4.80,131.0,205.0,178.0,276.4,27.0,65.42,174.0,61.81,131.0,56.67,619.0,0.25,1000.0,22.2,43.00,4.78,0.0,80.1,34.79,NaN,NaN,0
2,2016,2016-08-07,Travelers Championship,TPC River Highlands,Adam Hadwin,CUT,90,111.0,-0.011,55.0,0.223,121.0,-0.055,132.0,-0.081,11.0,0.530,22.0,21.74,16.0,3.01,48.0,4.03,68.0,4.66,35.0,141.0,92.0,290.1,49.0,63.62,83.0,66.07,97.0,58.27,193.0,0.91,1000.0,77.8,475.17,26.40,7.0,47.9,16.27,90.0,129.84,0
3,2016,2016-08-07,Travelers Championship,TPC River Highlands,Adam Rainaud,CUT,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,0.09,1000.0,NaN,0.00,NaN,NaN,90.0,NaN,NaN,NaN,0
4,2016,2016-08-07,Travelers Championship,TPC River Highlands,Alex Cejka,T11,11,39.0,0.631,148.0,-0.292,12.0,0.626,20.0,0.297,90.0,0.010,17.0,21.89,3.0,2.98,14.0,4.01,10.0,4.57,78.0,177.0,155.0,281.5,22.0,65.89,39.0,67.99,3.0,65.03,130.0,1.22,1000.0,60.0,445.75,29.72,2.0,53.0,19.12,90.0,129.84,1


#### Average the NaNs
At this point, the intentional offsetting of NaNs for poor players is over.  I don't want to penalize statistics or course history because you never know where they might land on the spectrum, so for these NaN values, we will take the mean.

In [11]:
# === Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in training_df.columns:
        training_df[col] = (
            training_df[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
            .astype(float)
        )

# Select only the numeric columns
numeric_columns = training_df.select_dtypes(include=['float64', 'int64'])

# Fill NaN values with the mean of each column
numeric_columns = numeric_columns.fillna(numeric_columns.mean())

# Update the original dataframe with the filled numeric columns
training_df.update(numeric_columns)

training_df.head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2016,2016-08-07,Travelers Championship,TPC River Highlands,Aaron Baddeley,T34,34,146.000000,-0.330000,159.00000,-0.366000,164.000000,-0.362000,10.000000,0.40200,8.000000,0.586000,59.000000,20.720000,78.000000,3.060000,14.000000,4.010000,50.00000,4.64000,159.000000,237.000000,82.000000,291.700000,155.000000,55.420000,154.000000,63.110000,5.000000,64.300000,143.0,1.14,60.0,58.800000,383.50,22.560000,1.000000,55.8,19.310000,48.000000,69.250000,0
1,2016,2016-08-07,Travelers Championship,TPC River Highlands,Abraham Ancer,T56,56,144.000000,-0.307000,111.00000,-0.024000,161.000000,-0.340000,77.000000,0.05700,172.000000,-0.421000,154.000000,18.630000,120.000000,3.080000,136.000000,4.070000,183.00000,4.80000,131.000000,205.000000,178.000000,276.400000,27.000000,65.420000,174.000000,61.810000,131.000000,56.670000,619.0,0.25,1000.0,22.200000,43.00,4.780000,0.000000,80.1,34.790000,59.369261,57.855034,0
2,2016,2016-08-07,Travelers Championship,TPC River Highlands,Adam Hadwin,CUT,90,111.000000,-0.011000,55.00000,0.223000,121.000000,-0.055000,132.000000,-0.08100,11.000000,0.530000,22.000000,21.740000,16.000000,3.010000,48.000000,4.030000,68.00000,4.66000,35.000000,141.000000,92.000000,290.100000,49.000000,63.620000,83.000000,66.070000,97.000000,58.270000,193.0,0.91,1000.0,77.800000,475.17,26.400000,7.000000,47.9,16.270000,90.000000,129.840000,0
3,2016,2016-08-07,Travelers Championship,TPC River Highlands,Adam Rainaud,CUT,90,98.712702,0.079046,96.30179,0.030489,98.456095,0.037964,99.223359,0.01009,96.296675,0.025311,98.664962,20.994066,88.678876,3.055434,90.310315,4.030503,95.40971,4.64155,96.075021,191.888321,97.174766,296.395482,94.713555,60.829335,98.652174,66.163248,96.127025,59.000716,1000.0,0.09,1000.0,56.981167,0.00,24.933518,1.464591,90.0,23.062179,59.369261,57.855034,0
4,2016,2016-08-07,Travelers Championship,TPC River Highlands,Alex Cejka,T11,11,39.000000,0.631000,148.00000,-0.292000,12.000000,0.626000,20.000000,0.29700,90.000000,0.010000,17.000000,21.890000,3.000000,2.980000,14.000000,4.010000,10.00000,4.57000,78.000000,177.000000,155.000000,281.500000,22.000000,65.890000,39.000000,67.990000,3.000000,65.030000,130.0,1.22,1000.0,60.000000,445.75,29.720000,2.000000,53.0,19.120000,90.000000,129.840000,1


In [12]:
# Check for NaN values
assert training_df.isna().sum().sum() == 0, "🚨 Still missing values!"

In [13]:
# Inspect which columns have NaN values
training_df.isna().sum().sort_values(ascending=False).head(10)


SEASON         0
ENDING_DATE    0
TOURNAMENT     0
COURSE         0
PLAYER         0
POS            0
FINAL_POS      0
SGTTG_RANK     0
SGTTG          0
SGOTT_RANK     0
dtype: int64

### Check Features for Outliers and General Health (Histograms)
Now that all the data is cleaned, let's look at the distribution graphs.

In [ ]:
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Select numeric columns with >1 unique value, excluding label
df_numeric = training_df.select_dtypes(include=np.number)
df_numeric = df_numeric.loc[:, df_numeric.nunique() > 1]
df_numeric = df_numeric.drop(columns=["TOP_20"], errors="ignore")

# Create subplots
n_cols = 3
n_rows = int(np.ceil(len(df_numeric.columns) / n_cols))
fig = sp.make_subplots(rows=n_rows, cols=n_cols, subplot_titles=df_numeric.columns)

# Add one histogram per feature with smart binning
for i, column in enumerate(df_numeric.columns):
    row = i // n_cols + 1
    col = i % n_cols + 1

    # Compute dynamic bin count
    col_range = df_numeric[column].max() - df_numeric[column].min()
    n_unique = df_numeric[column].nunique()
    nbins = min(50, max(5, int(n_unique / 2))) if col_range < 10 else 50  # tighter for short-range

    fig.add_trace(
        go.Histogram(
            x=df_numeric[column],
            nbinsx=nbins,
            histnorm="density",
            marker=dict(color="#636EFA"),
            opacity=0.8
        ),
        row=row,
        col=col
    )

# Style the figure
fig.update_layout(
    height=300 * n_rows,
    width=1000,
    title_text="Feature Distributions (with Adaptive Binning)",
    template="plotly_dark",
    showlegend=False
)

fig.show()

# Current Week Data

## Import 2025 Field
**Important!**  Save curren't week's `DKSalaries.csv` into Data folder before this step.

In [14]:
importlib.reload(utils.db_utils)
from utils.db_utils import DK_PLAYER_NAME_MAP

# Load DraftKings player list
dk = pd.read_csv("data/DKSalaries.csv", usecols=["Name", "Salary"])

# Normalize DK names to PGA naming convention
dk["Name"] = dk["Name"].replace(DK_PLAYER_NAME_MAP)

# Rename for consistency
dk = dk.rename(columns={"Name": "PLAYER", "Salary": "SALARY"})

dk.head(10)

,PLAYER,SALARY
0,Scottie Scheffler,13300
1,Rory McIlroy,11500
2,Xander Schauffele,10800
3,Collin Morikawa,10200
4,Patrick Cantlay,9800
5,Ludvig Aberg,9600
6,Justin Thomas,9300
7,Sepp Straka,9200
8,Sam Burns,9100
9,Viktor Hovland,9000


## Vegas Odds
Confirm the [Vegas Odds](http://golfodds.com/weekly-odds.html) page is updated with current tournament.

Note: sometimes this site can give errors and not load correctly.  Just have to do it again later.
Another trick to try is change the url try to run it and then put the url back the way it was.  Worked once, maybe random coincidence.

In [15]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_current_week_odds

odds_current = get_current_week_odds(season=2025, tournament_name=new_tournament_name)
odds_current.head(10)


,SEASON,TOURNAMENT,PLAYER,ODDS,VEGAS_ODDS
1,2025,Travelers Championship,Scottie Scheffler,3/1,3.0
2,2025,Travelers Championship,Rory McIlroy,14/1,14.0
3,2025,Travelers Championship,Collin Morikawa,16/1,16.0
4,2025,Travelers Championship,Xander Schauffele,16/1,16.0
5,2025,Travelers Championship,Patrick Cantlay,25/1,25.0
6,2025,Travelers Championship,Ludvig Aberg,30/1,30.0
7,2025,Travelers Championship,Justin Thomas,35/1,35.0
8,2025,Travelers Championship,Viktor Hovland,35/1,35.0
9,2025,Travelers Championship,Sepp Straka,30/1,30.0
10,2025,Travelers Championship,Tommy Fleetwood,30/1,30.0


## Cut Percentage and FedEx Points

In [16]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts_rolling = get_cut_and_fedex_history(db_path, this_week_history, window_months=9)
cuts_current = cuts_rolling[this_week_key].copy()
cuts_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,A.J. Ewart,1,0,0.000,0.0,0.00,0,2025-06-22,Travelers Championship
1,Aaron Baddeley,15,6,49.088,40.0,3.27,0,2025-06-22,Travelers Championship
2,Aaron Rai,16,12,605.730,75.0,37.86,1,2025-06-22,Travelers Championship
3,Aaron Wise,1,0,0.000,0.0,0.00,0,2025-06-22,Travelers Championship
4,Adam Hadwin,17,11,203.569,64.7,11.97,2,2025-06-22,Travelers Championship
5,Adam Long,1,0,0.000,0.0,0.00,0,2025-06-22,Travelers Championship
6,Adam Schenk,19,9,270.904,47.4,14.26,2,2025-06-22,Travelers Championship
7,Adam Scott,12,10,361.953,83.3,30.16,5,2025-06-22,Travelers Championship
8,Adam Svensson,19,10,126.043,52.6,6.63,0,2025-06-22,Travelers Championship
9,Adrien Dumont de Chassart,7,3,200.500,42.9,28.64,0,2025-06-22,Travelers Championship


## Recent Form

In [17]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish(db_path, this_week_history, window_months=9)
recent_form_current = recent_form[this_week_key].copy()

recent_form_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Carlos Ortiz,1,4.0,5.77,2025-06-22,Travelers Championship
1,Scottie Scheffler,14,6.9,2.55,2025-06-22,Travelers Championship
2,Jon Rahm,3,9.7,7.00,2025-06-22,Travelers Championship
3,Bubba Watson,1,14.0,20.20,2025-06-22,Travelers Championship
4,Ian Gilligan,1,16.0,23.08,2025-06-22,Travelers Championship
5,Taisei Shimizu,1,16.0,23.08,2025-06-22,Travelers Championship
6,Stewart Cink,1,17.0,24.53,2025-06-22,Travelers Championship
7,Rory McIlroy,10,20.3,8.47,2025-06-22,Travelers Championship
8,Ren Yonezawa,1,22.0,31.74,2025-06-22,Travelers Championship
9,Yuto Katsuragawa,1,22.0,31.74,2025-06-22,Travelers Championship


## Course History

In [18]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

course_hist = get_course_history(db_path, this_week_history, lookback_years=7)
course_hist_current = course_hist[this_week_key].copy()

course_hist_current.head(10)


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,3,90.0,64.92,2025-06-22,TPC River Highlands,Travelers Championship
1,Aaron Rai,2,32.0,29.13,2025-06-22,TPC River Highlands,Travelers Championship
2,Aaron Wise,3,67.3,48.55,2025-06-22,TPC River Highlands,Travelers Championship
3,Abraham Ancer,4,28.3,17.58,2025-06-22,TPC River Highlands,Travelers Championship
4,Adam D'Amario,2,90.0,81.92,2025-06-22,TPC River Highlands,Travelers Championship
5,Adam Hadwin,4,62.3,38.71,2025-06-22,TPC River Highlands,Travelers Championship
6,Adam Long,5,50.0,27.91,2025-06-22,TPC River Highlands,Travelers Championship
7,Adam Schenk,7,85.0,40.88,2025-06-22,TPC River Highlands,Travelers Championship
8,Adam Scott,3,23.7,17.10,2025-06-22,TPC River Highlands,Travelers Championship
9,Adam Svensson,4,55.3,34.36,2025-06-22,TPC River Highlands,Travelers Championship


## Merged Dataframe

In [19]:
importlib.reload(utils.db_utils)
from utils.db_utils import build_test_rows

this_week = build_test_rows(
    db_path=db_path,
    stats_df=stats_df,
    odds_df=odds_current,
    cuts_df=cuts_current,
    recent_form_df=recent_form_current,
    course_hist_df=course_hist_current,
    dk_df=dk,
    season=tournament_config["new"]["season"]
)

this_week.head()

,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,SALARY
0,Aaron Rai,22.0,0.815,25.0,0.401,31.0,0.414,97.0,0.001,125.0,-0.137,54.0,22.22%,174.0,3.20,12.0,3.98,30.0,4.55,79.0,170,169.0,286.9,1.0,73.55%,18.0,69.12%,123.0,57.79%,32.0,2.6753,2025.0,45.0,75.0,605.730,37.86,1.0,39.7,14.01,32.0,29.13,7500
1,Adam Hadwin,150.0,-0.496,142.0,-0.252,153.0,-0.383,57.0,0.139,113.0,-0.066,48.0,22.33%,21.0,3.01,134.0,4.06,137.0,4.66,98.0,180,106.0,299.0,74.0,60.25%,139.0,63.29%,163.0,54.60%,103.0,1.2562,2025.0,500.0,64.7,203.569,11.97,2.0,61.1,21.14,62.3,38.71,6100
2,Adam Scott,56.0,0.402,34.0,0.319,112.0,-0.084,49.0,0.167,96.0,0.014,77.0,21.72%,154.0,3.13,134.0,4.06,41.0,4.57,122.0,199,83.0,302.0,116.0,56.98%,124.0,63.89%,139.0,56.64%,40.0,2.4827,2025.0,80.0,83.3,361.953,30.16,5.0,41.0,15.98,23.7,17.10,7600
3,Akshay Bhatia,77.0,0.213,105.0,0.003,25.0,0.456,150.0,-0.247,14.0,0.474,7.0,24.57%,5.0,2.98,44.0,4.01,157.0,4.70,136.0,211,154.0,293.6,57.0,61.40%,65.0,66.03%,160.0,55.03%,34.0,2.6269,2025.0,70.0,75.0,800.240,50.02,0.0,39.9,14.08,5.0,7.21,7400
4,Alex Noren,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,1.6465,2025.0,100.0,75.0,104.000,26.00,0.0,49.3,30.63,NaN,NaN,6500


### Dataframe Normalization

Run the same normalization techniques that I ran on the historical data:
- Any NaN Odds go to 1000/1 and clip all values at 1000/1
- OWGR NaN values go to 1000
- Recent form NaN values act like a MC at 90
- Average everything else.

In [21]:
# === 1. Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in this_week.columns:
        this_week[col] = (
            this_week[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
        )
        this_week[col] = pd.to_numeric(this_week[col], errors="coerce")

# === 2. Normalize & assign fallback values for key fields ===
this_week["VEGAS_ODDS"] = pd.to_numeric(this_week["VEGAS_ODDS"], errors="coerce").fillna(1000).clip(upper=1000)

if "OWGR" in this_week.columns:
    this_week["OWGR"] = pd.to_numeric(this_week["OWGR"], errors="coerce")
    max_owgr = this_week["OWGR"].dropna().max()
    this_week["OWGR"] = this_week["OWGR"].fillna(max_owgr).clip(upper=1000)

if "OWGR_RANK" in this_week.columns:
    this_week["OWGR_RANK"] = pd.to_numeric(this_week["OWGR_RANK"], errors="coerce").fillna(1000).clip(upper=1000)

this_week["RECENT_FORM"] = pd.to_numeric(this_week["RECENT_FORM"], errors="coerce").fillna(90)
this_week["FEDEX_CUP_POINTS"] = pd.to_numeric(this_week["FEDEX_CUP_POINTS"], errors="coerce").fillna(0)

if "COURSE_HISTORY" in this_week.columns:
    this_week["COURSE_HISTORY"] = pd.to_numeric(this_week["COURSE_HISTORY"], errors="coerce")
    ch_mean = this_week["COURSE_HISTORY"].mean()
    this_week["COURSE_HISTORY"] = this_week["COURSE_HISTORY"].fillna(ch_mean)

# === 3. Fill all remaining NaNs in numeric columns ===
# Recast everything that might look numeric to be sure
for col in this_week.columns:
    if col not in ["PLAYER", "SALARY", "TOURNAMENT", "SEASON"]:
        try:
            this_week[col] = pd.to_numeric(this_week[col], errors="coerce")
        except Exception:
            continue

numeric_cols = this_week.select_dtypes(include=["number"]).columns.tolist()

for col in numeric_cols:
    if this_week[col].isna().any():
        col_mean = this_week[col].mean()
        this_week[col] = this_week[col].fillna(col_mean if not np.isnan(col_mean) else 0)


In [22]:
# === Final Check: No NaNs Should Remain ===
assert this_week.isna().sum().sum() == 0, "🚨 Still missing values in prediction set!"

In [23]:
# === Check for Remaining Missing Values ===
missing_summary = this_week.isna().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)

print("🧹 Columns still containing NaN values:")
display(missing_summary)

🧹 Columns still containing NaN values:


Series([], dtype: int64)

In [24]:
this_week.head(10)

,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,SALARY
0,Aaron Rai,22.000000,0.815000,25.000000,0.401000,31.000000,0.41400,97.000000,0.001000,125.000000,-0.137000,54.000000,22.220000,174.000000,3.200000,12.000000,3.980000,30.000000,4.550000,79.000000,170.000000,169.000000,286.900000,1.000000,73.550000,18.00000,69.120000,123.00000,57.790000,32.0,2.6753,2025.0,45.0,75.0,605.730,37.86,1.0,39.7,14.01,32.000000,29.130000,7500
1,Adam Hadwin,150.000000,-0.496000,142.000000,-0.252000,153.000000,-0.38300,57.000000,0.139000,113.000000,-0.066000,48.000000,22.330000,21.000000,3.010000,134.000000,4.060000,137.000000,4.660000,98.000000,180.000000,106.000000,299.000000,74.000000,60.250000,139.00000,63.290000,163.00000,54.600000,103.0,1.2562,2025.0,500.0,64.7,203.569,11.97,2.0,61.1,21.14,62.300000,38.710000,6100
2,Adam Scott,56.000000,0.402000,34.000000,0.319000,112.000000,-0.08400,49.000000,0.167000,96.000000,0.014000,77.000000,21.720000,154.000000,3.130000,134.000000,4.060000,41.000000,4.570000,122.000000,199.000000,83.000000,302.000000,116.000000,56.980000,124.00000,63.890000,139.00000,56.640000,40.0,2.4827,2025.0,80.0,83.3,361.953,30.16,5.0,41.0,15.98,23.700000,17.100000,7600
3,Akshay Bhatia,77.000000,0.213000,105.000000,0.003000,25.000000,0.45600,150.000000,-0.247000,14.000000,0.474000,7.000000,24.570000,5.000000,2.980000,44.000000,4.010000,157.000000,4.700000,136.000000,211.000000,154.000000,293.600000,57.000000,61.400000,65.00000,66.030000,160.00000,55.030000,34.0,2.6269,2025.0,70.0,75.0,800.240,50.02,0.0,39.9,14.08,5.000000,7.210000,7400
4,Alex Noren,64.246377,0.378493,78.130435,0.098913,69.855072,0.19042,71.869565,0.089319,71.101449,0.142145,71.275362,22.080725,86.318841,3.066377,72.072464,4.023043,87.869565,4.614928,86.202899,175.536232,94.623188,300.485507,80.913043,60.098261,91.15942,65.284783,96.15942,59.245217,75.0,1.6465,2025.0,100.0,75.0,104.000,26.00,0.0,49.3,30.63,49.642623,35.744426,6500
5,Andrew Novak,49.000000,0.462000,71.000000,0.136000,77.000000,0.09100,31.000000,0.235000,60.000000,0.167000,117.000000,20.620000,164.000000,3.140000,103.000000,4.040000,60.000000,4.590000,151.000000,223.000000,105.000000,299.100000,118.000000,56.950000,166.00000,60.830000,54.00000,61.540000,35.0,2.5962,2025.0,100.0,62.5,1532.225,63.84,3.0,46.5,14.45,68.000000,61.900000,6700
6,Austin Eckroat,136.000000,-0.309000,55.000000,0.217000,114.000000,-0.09500,166.000000,-0.431000,119.000000,-0.121000,117.000000,20.620000,92.000000,3.070000,149.000000,4.070000,159.000000,4.710000,58.000000,159.000000,128.000000,297.400000,31.000000,63.970000,135.00000,63.460000,162.00000,54.680000,73.0,1.6916,2025.0,250.0,60.0,254.542,12.73,4.0,58.0,19.05,32.700000,23.590000,6100
7,Ben Griffin,25.000000,0.755000,53.000000,0.238000,37.000000,0.35400,50.000000,0.164000,41.000000,0.252000,53.000000,22.300000,29.000000,3.020000,44.000000,4.010000,41.000000,4.570000,64.000000,162.000000,67.000000,304.300000,95.000000,58.920000,30.00000,68.630000,105.00000,59.110000,15.0,3.9504,2025.0,55.0,80.8,1949.496,74.98,5.0,36.4,11.04,78.500000,71.450000,8100
8,Brian Campbell,143.000000,-0.416000,173.000000,-1.033000,43.000000,0.31600,19.000000,0.301000,73.000000,0.114000,102.000000,21.210000,51.000000,3.040000,160.000000,4.090000,137.000000,4.660000,121.000000,197.000000,176.000000,276.100000,21.000000,65.740000,152.00000,62.250000,144.00000,56.520000,111.0,1.1999,2025.0,600.0,43.8,603.535,37.72,0.0,66.4,23.44,49.642623,35.744426,6000
9,Brian Harman,89.000000,0.058000,121.000000,-0.054000,95.000000,0.01000,65.000000,0.102000,103.000000,-0.002000,135.000000,20.110000,117.000000,3.090000,10

## Correlations
**Last 4 Years**

In [25]:
import plotly.express as px

# Get the 4 most recent seasons (or fewer if not available)
available_seasons = sorted(training_df["SEASON"].unique())[-4:]

# Dictionary to store correlation results
correlation_frames = {}

# Calculate correlations for each season
for season in available_seasons:
    df = training_df[training_df["SEASON"] == season].copy()

    # Numeric features only, drop final result
    numeric = df.select_dtypes(include=["float64", "int64"]).drop(columns=["FINAL_POS", "SEASON"], errors="ignore")

    # Skip if TOP_20 is not present
    if "TOP_20" not in numeric.columns:
        continue

    # Correlation with TOP_20
    cor = numeric.corr().abs()["TOP_20"].drop("TOP_20", errors="ignore").sort_values(ascending=False)
    cor_df = cor.reset_index()
    cor_df.columns = ["Feature", "Correlation"]
    correlation_frames[season] = cor_df

# Plot one bar chart per season
for season, cor_df in correlation_frames.items():
    fig = px.bar(
        cor_df,
        x="Feature",
        y="Correlation",
        title=f"🔎 Correlation to TOP_20 — Season {int(season)}",
        height=500,
        width=1000
    )

    fig.update_layout(
        xaxis_title="Feature",
        yaxis_title="Absolute Correlation to TOP_20",
        template="plotly_dark",
        xaxis_tickangle=-45
    )

    fig.show()


**Average of Last 4 Years**

In [26]:
import pandas as pd
import numpy as np
import plotly.express as px

# === Rebuild correlation matrices for the 4 most recent seasons ===
cor = {}
recent_seasons = sorted(training_df["SEASON"].unique())[-4:]

for season in recent_seasons:
    df_season = training_df[training_df["SEASON"] == season].copy()
    numeric_cols = df_season.select_dtypes(include=["float64", "int64"])
    cor[int(season)] = numeric_cols.corr().abs()

# === Combine into one summary DataFrame ===
cor_df = pd.DataFrame()

for season in recent_seasons:
    df = cor[season].reset_index()
    if "TOP_20" not in df.columns:
        continue
    temp = df[["index", "TOP_20"]].copy()
    temp = temp.rename(columns={"TOP_20": str(season)})
    if cor_df.empty:
        cor_df = temp
    else:
        cor_df = cor_df.merge(temp, on="index", how="outer")

# Drop target and metadata rows
cor_df = cor_df[~cor_df["index"].isin(["TOP_20", "FINAL_POS", "SEASON"])]

# Compute average correlation across seasons
cor_df["AVERAGE"] = cor_df.drop(columns=["index"]).mean(axis=1)
cor_df = cor_df.sort_values(by="AVERAGE", ascending=False)

# === Plotly Bar Chart ===
fig = px.bar(
    cor_df,
    x="index",
    y="AVERAGE",
    title="Average Correlation to TOP_20 (Last 4 Seasons)",
    labels={"index": "Feature", "AVERAGE": "Avg Correlation"},
    template="plotly_dark"
)

fig.update_layout(
    xaxis_tickangle=-45,
    height=600,
    width=1000,
    showlegend=False
)

fig.show()


# ML Model

## Predictors

In [27]:
# === Define Features and Target ===
target_col = "TOP_20"

# Columns to exclude from features
exclude = [
    "PLAYER", "TOURNAMENT", "COURSE", "ENDING_DATE", "SEASON", "TOURN_ID",
    "TOP_20", "FINAL_POS"  
]

# Select numeric feature columns
feature_cols = [
    col for col in training_df.columns
    if col not in exclude and training_df[col].dtype in [np.float64, np.int64]
]

print(f"Selected {len(feature_cols)} features:\n", feature_cols)

Selected 39 features:
 ['SGTTG_RANK', 'SGTTG', 'SGOTT_RANK', 'SGOTT', 'SGAPR_RANK', 'SGAPR', 'SGATG_RANK', 'SGATG', 'SGP_RANK', 'SGP', 'BIRDIES_RANK', 'BIRDIES', 'PAR_3_RANK', 'PAR_3', 'PAR_4_RANK', 'PAR_4', 'PAR_5_RANK', 'PAR_5', 'TOTAL_DRIVING_RANK', 'TOTAL_DRIVING', 'DRIVING_DISTANCE_RANK', 'DRIVING_DISTANCE', 'DRIVING_ACCURACY_RANK', 'DRIVING_ACCURACY', 'GIR_RANK', 'GIR', 'SCRAMBLING_RANK', 'SCRAMBLING', 'OWGR_RANK', 'OWGR', 'VEGAS_ODDS', 'CUT_PERCENTAGE', 'FEDEX_CUP_POINTS', 'form_density', 'CONSECUTIVE_CUTS', 'RECENT_FORM', 'adj_form', 'COURSE_HISTORY', 'adj_ch']


### Random Forest
Initial default model before hyperparameter tuning.

This one actually performed really well (ROC AUC above 0.85).  Strangely doing hyperparameter tuning hurt the model (less than 0.7), so I removed that step and we will just use this default model.  This does cross-validation testing on 5 folds to ensure robust train/test splits.

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# === Standardize features ===
scaler = StandardScaler()
X_rf_std = scaler.fit_transform(training_df[feature_cols])
y_rf = training_df["TOP_20"]

# === Balance the dataset with SMOTE and undersampling ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
pipeline = Pipeline([("o", over), ("u", under)])
X_rf_bal, y_rf_bal = pipeline.fit_resample(X_rf_std, y_rf)

# === Define and evaluate Random Forest ===
rf = RandomForestClassifier(n_estimators=100, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf, X_rf_bal, y_rf_bal, cv=cv, scoring="roc_auc")

print("📊 Random Forest (All Features)")
print("Cross-validation scores:", scores)
print("Average ROC AUC score: {:.2f}".format(scores.mean()))


📊 Random Forest (All Features)
Cross-validation scores: [0.94702135 0.94710251 0.94432842 0.92370878 0.89802699]
Average ROC AUC score: 0.93


Random Forest Training & Feature Importance

Now we train on all the data (no longer cross-validation folds) and extract the feature importance to see how it compares with linear correlations.


In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import plotly.express as px

# === Resample the full dataset ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
resample_pipeline = Pipeline([("o", over), ("u", under)])
X_all_resampled, y_all_resampled = resample_pipeline.fit_resample(
    training_df[feature_cols], training_df["TOP_20"]
)

# === Standardize ===
scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all_resampled)

# === Train Calibrated Random Forest ===
base_rf = RandomForestClassifier(random_state=42)
final_model = CalibratedClassifierCV(estimator=base_rf, method="sigmoid", cv=5)
final_model.fit(X_all_scaled, y_all_resampled)

# === Aggregate feature importances across all CV folds ===
all_importances = np.array([
    est.estimator.feature_importances_ for est in final_model.calibrated_classifiers_
])
mean_importance = all_importances.mean(axis=0)

importance_df = pd.DataFrame({
    "Feature": feature_cols,
    "Importance": mean_importance
}).sort_values(by="Importance", ascending=False)

# === Plot feature importances ===
fig = px.bar(
    importance_df,
    x="Feature",
    y="Importance",
    title="Random Forest Feature Importances",
    template="plotly_dark"
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()


Use this model on this week's data to predict the best performers.

In [30]:
# === Prepare test data (this week's players) ===

# Select and standardize features using the same scaler as training
X_test = this_week[feature_cols]
X_test_scaled = scaler.transform(X_test)

# === Predict probabilities for Top 20 finish ===
# CalibratedClassifierCV returns well-calibrated probabilities
this_week["PROBABILITY"] = final_model.predict_proba(X_test_scaled)[:, 1]

# === Sort results by highest predicted probability ===
this_week_sorted = this_week.sort_values(by="PROBABILITY", ascending=False).reset_index(drop=True)

# === Define final column order explicitly ===
columns_to_show = [
    "PLAYER", "SALARY", "PROBABILITY",
    "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", "CUT_PERCENTAGE", 
    "OWGR_RANK", "FEDEX_CUP_POINTS",

    # SG stats
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",

    # Scoring/Performance
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",

    # Driving / Ball Striking
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Keep only columns that exist in the dataframe
columns_to_show = [col for col in columns_to_show if col in this_week_sorted.columns]

# Final dataframe for display/export
export_df = this_week_sorted[columns_to_show].copy()

# Preview top 20
export_df.head(20)


,PLAYER,SALARY,PROBABILITY,RECENT_FORM,COURSE_HISTORY,VEGAS_ODDS,CUT_PERCENTAGE,OWGR_RANK,FEDEX_CUP_POINTS,SGTTG_RANK,SGOTT_RANK,SGAPR_RANK,SGATG_RANK,SGP_RANK,BIRDIES_RANK,PAR_3_RANK,PAR_4_RANK,PAR_5_RANK,TOTAL_DRIVING_RANK,DRIVING_DISTANCE_RANK,DRIVING_ACCURACY_RANK,GIR_RANK,SCRAMBLING_RANK
0,Scottie Scheffler,13300,0.956233,6.9,31.0,3.0,100.0,1.0,3725.967,1.0,2.0,1.0,29.0,20.0,3.0,21.0,1.0,2.0,16.0,73.0,34.0,10.0,2.0
1,Ben Griffin,8100,0.925624,36.4,78.5,55.0,80.8,15.0,1949.496,25.0,53.0,37.0,50.0,41.0,53.0,29.0,44.0,41.0,64.0,67.0,95.0,30.0,105.0
2,Maverick McNealy,7600,0.902446,37.0,60.0,60.0,78.3,14.0,2214.922,48.0,43.0,41.0,133.0,37.0,46.0,128.0,44.0,73.0,79.0,72.0,98.0,43.0,149.0
3,Daniel Berger,7500,0.872361,34.7,78.5,45.0,81.8,29.0,1489.533,7.0,19.0,21.0,59.0,105.0,96.0,128.0,17.0,132.0,13.0,92.0,12.0,39.0,41.0
4,Rory McIlroy,11500,0.859232,20.3,12.3,14.0,90.0,2.0,2694.036,8.0,1.0,79.0,43.0,12.0,18.0,39.0,17.0,16.0,78.0,2.0,167.0,68.0,33.0
5,Corey Conners,8700,0.856233,32.6,54.0,45.0,87.5,24.0,1465.490,31.0,14.0,47.0,116.0,59.0,68.0,80.0,29.0,73.0,43.0,139.0,11.0,22.0,70.0
6,Patrick Cantlay,9800,0.848800,27.9,10.9,25.0,85.7,20.0,1135.952,11.0,31.0,15.0,78.0,65.0,14.0,128.0,6.0,73.0,53.0,75.0,81.0,4.0,139.0
7,Collin Morikawa,10200,0.832836,23.3,57.3,16.0,100.0,4.0,1334.158,2.0,12.0,4.0,73.0,110.0,9.0,2.0,29.0,93.0,58.0,156.0,3.0,34.0,81.0
8,J.J. Spaun,8600,0.831118,36.8,65.7,50.0,78.3,8.0,2164.437,10.0,47.0,7.0,104.0,40.0,57.0,51.0,17.0,73.0,23.0,63.0,56.0,35.0,106.0
9,Keegan Bradley,8500,0.792450,26.5,41.1,35.0,93.3,21.0,1001.967,4.0,26.0,28.0,14.0,132.0,58.0,105.0,17.0,120.0,19.0,68.0,45.0,74.0,118.0


### Save to CSV

In [31]:
# Ensure required columns for spreadsheet export
if "COURSE_HISTORY" not in export_df.columns:
    export_df["COURSE_HISTORY"] = "-"

# Define columns to round (FedEx + stats)
columns_to_round = [
    "FEDEX_CUP_POINTS",
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

columns_to_round_1 = [
    "COURSE_HISTORY", "CUT_PERCENTAGE"
]

# Round only the selected columns to 0 decimal places
export_df[columns_to_round] = export_df[columns_to_round].round(0).astype("Int64")
# Round only the selected columns to 1 decimal places
export_df[columns_to_round_1] = export_df[columns_to_round_1].round(1)

# Explicit column order for Excel integration
column_order = [
    "PLAYER", "SALARY", "PROBABILITY", "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", 
    "CUT_PERCENTAGE", "OWGR_RANK", "FEDEX_CUP_POINTS", 
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Ensure all expected columns exist
for col in column_order:
    if col not in export_df.columns:
        export_df[col] = "-" if col == "COURSE_HISTORY" else np.nan

# Reorder columns to match Excel layout
export_df = export_df[column_order]

# Export to CSV
filename = f"data/current_week_export.csv"
export_df.to_csv(filename, index=False)
print(f"✅ Exported to {filename}")

✅ Exported to data/current_week_export.csv
